In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bpc1-data/BPc1 15min.csv
/kaggle/input/bpc1-data/BPc1 30min.csv
/kaggle/input/bpc1-data/BPc1 5min.csv
/kaggle/input/bpc1-data/BPc1 1 day.csv
/kaggle/input/bpc1-data/BPc1 1hr.csv
/kaggle/input/bpc1-data/BPc1 1min.csv
/kaggle/input/adc1-data/ADc1_15 minutes.csv
/kaggle/input/adc1-data/ADc1_30 minutes.csv
/kaggle/input/adc1-data/ADc1_1 Hour.csv
/kaggle/input/adc1-data/ADc1_day.csv
/kaggle/input/adc1-data/ADc1_1 minute.csv
/kaggle/input/adc1-data/ADc1_4 Hour.csv
/kaggle/input/adc1-data/ADc1_5 minutes.csv
/kaggle/input/esc1-data/ESc1_5 minutes.csv
/kaggle/input/esc1-data/ESc1_15 minutes.csv
/kaggle/input/esc1-data/ESc1_1 Hour.csv
/kaggle/input/esc1-data/ESc1_30 minutes.csv
/kaggle/input/esc1-data/ESc1_1 minute.csv
/kaggle/input/esc1-data/ESc1_1 Day.csv
/kaggle/input/uroc1-data/UROc1_BTT.csv
/kaggle/input/uroc1-data/UROc1 1 day.csv
/kaggle/input/uroc1-data/UROc1 5min.csv
/kaggle/input/uroc1-data/UROc1 30min.csv
/kaggle/input/uroc1-data/UROc1 1hr.csv
/kaggle/input/uroc1-data/URO

In [4]:
import statsmodels.api as sm
#import talib as ta
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date
from statsmodels.tsa.stattools import coint, adfuller
import copy

In [5]:
ad = pd.read_csv('/kaggle/input/adc1-data/ADc1_30 minutes.csv')

In [6]:
ad

,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC
0,01/02/2011 23:30,1.0132,1.01370,1.01170,1.01180,688,ADH1
1,01/03/2011 00:00,1.0118,1.01290,1.01150,1.01250,487,ADH1
2,01/03/2011 00:30,1.0126,1.01350,1.01150,1.01190,645,ADH1
3,01/03/2011 01:00,1.0119,1.01220,1.01100,1.01150,693,ADH1
4,01/03/2011 01:30,1.0115,1.01150,1.00970,1.00970,1203,ADH1
...,...,...,...,...,...,...,...
110043,03/10/2023 22:00,0.6577,0.65800,0.65770,0.65800,12,ADH3
110044,03/12/2023 22:30,0.6596,0.66290,0.65960,0.66290,13,ADH3
110045,03/12/2023 23:00,0.6618,0.66180,0.66180,0.66180,1,ADH3
110046,03/12/2023 23:30,0.6626,0.66480,0.66215,0.66365,159,ADH3


In [7]:
ad = ad.dropna()
ad.Timestamp  = pd.to_datetime(ad.Timestamp, format = '%m/%d/%Y %H:%M')

In [8]:

#Select date range
start_rem= pd.to_datetime("2021-03-13 05:30:00")
stop_rem =pd.to_datetime("2023-03-13 05:30:00")

# S&P500 train
#Slicing list of trades
after_sm1 = ad["Timestamp"] >= start_rem
before_sm1 = ad["Timestamp"] <= stop_rem
between_sm1 = after_sm1 & before_sm1
sm1f = ad.loc[between_sm1]

ad = sm1f
ad = ad.reset_index(drop=True)
ad


,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1
...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3


In [9]:
ad

,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1
...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3


In [10]:


def calculate_rsi(data, column='Close', period=14):
    """
    Calculate the Relative Strength Index (RSI) for a given dataset.

    Parameters:
    - data: Pandas DataFrame containing the price data.
    - column: Name of the column to calculate RSI on (default is 'Close').
    - period: Lookback period for RSI calculation (default is 14).

    Returns:
    - Pandas Series containing RSI values.
    """
    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")

    delta = data[column].diff(1)  # Price differences

    gain = np.where(delta > 0, delta, 0)  # Positive gains
    loss = np.where(delta < 0, -delta, 0)  # Negative losses

    avg_gain = pd.Series(gain, index=data.index).rolling(window=period, min_periods=period).mean()
    avg_loss = pd.Series(loss, index=data.index).rolling(window=period, min_periods=period).mean()

    rs = avg_gain / (avg_loss + 1e-10)  # Adding a small value to avoid division by zero
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Example usage:
ad['RSI'] = calculate_rsi(ad, column='Close', period=14)


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


In [11]:
ad

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC,RSI
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1,NaN
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1,NaN
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1,NaN
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1,NaN
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1,NaN
...,...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3,27.222218
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3,51.838230
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3,45.551597
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3,55.743238


In [12]:

def calculate_bbands(data, column='Close', period=20, nbdevup=2, nbdevdn=2):
    """
    Calculate Bollinger Bands for a given dataset.

    Parameters:
    - data: Pandas DataFrame containing the price data.
    - column: Name of the column to calculate Bollinger Bands on (default is 'Close').
    - period: Lookback period for the moving average (default is 20).
    - nbdevup: Number of standard deviations for the upper band (default is 2).
    - nbdevdn: Number of standard deviations for the lower band (default is 2).

    Returns:
    - Three Pandas Series: Upper Band, Middle Band (SMA), Lower Band.
    """
    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")

    middle_band = data[column].rolling(window=period, min_periods=period).mean()
    std_dev = data[column].rolling(window=period, min_periods=period).std()

    upper_band = middle_band + (std_dev * nbdevup)
    lower_band = middle_band - (std_dev * nbdevdn)

    return upper_band, middle_band, lower_band

ad["Upper_Band"], ad["Middle_Band"], ad["Lower_Band"] = calculate_bbands(ad, column="Close", period=20, nbdevup=2, nbdevdn=2)


In [13]:

def calculate_momentum(data, column='Close', period=14):
    """
    Calculate Momentum (MOM) indicator for a given dataset.

    Parameters:
    - data: Pandas DataFrame containing the price data.
    - column: Name of the column to calculate MOM on (default is 'Close').
    - period: Lookback period for MOM calculation (default is 14).

    Returns:
    - Pandas Series containing Momentum values.
    """
    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    return data[column] - data[column].shift(period)


ad['MOM'] = calculate_momentum(ad, column='Close', period=14)


In [14]:
ad

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC,RSI,Upper_Band,Middle_Band,Lower_Band,MOM
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1,NaN,NaN,NaN,NaN,NaN
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1,NaN,NaN,NaN,NaN,NaN
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1,NaN,NaN,NaN,NaN,NaN
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1,NaN,NaN,NaN,NaN,NaN
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3,27.222218,0.663573,0.659915,0.656257,-0.00410
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3,51.838230,0.663970,0.660120,0.656270,0.00050
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3,45.551597,0.664144,0.660253,0.656361,-0.00125
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3,55.743238,0.664614,0.660475,0.656336,0.00170


In [15]:

def calculate_bbands(data, column='RSI', period=20, nbdevup=2, nbdevdn=2):
    """
    Calculate Bollinger Bands for a given dataset (applied to RSI or any other indicator).

    Parameters:
    - data: Pandas DataFrame containing the RSI or other indicator.
    - column: Name of the column to calculate Bollinger Bands on (default is 'RSI').
    - period: Lookback period for the moving average (default is 20).
    - nbdevup: Number of standard deviations for the upper band (default is 2).
    - nbdevdn: Number of standard deviations for the lower band (default is 2).

    Returns:
    - Three Pandas Series: Upper Band, Middle Band (SMA), Lower Band.
    """
    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")

    middle_band = data[column].rolling(window=period, min_periods=period).mean()
    std_dev = data[column].rolling(window=period, min_periods=period).std()

    upper_band = middle_band + (std_dev * nbdevup)
    lower_band = middle_band - (std_dev * nbdevdn)

    return upper_band, middle_band, lower_band


ad["RSI_UB"], ad["RSI_MB"], ad["RSI_LB"] = calculate_bbands(ad, column="RSI", period=20, nbdevup=2, nbdevdn=2)


In [16]:
ad

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC,RSI,Upper_Band,Middle_Band,Lower_Band,MOM,RSI_UB,RSI_MB,RSI_LB
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3,27.222218,0.663573,0.659915,0.656257,-0.00410,90.769799,56.535307,22.300816
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3,51.838230,0.663970,0.660120,0.656270,0.00050,90.793162,56.788510,22.783857
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3,45.551597,0.664144,0.660253,0.656361,-0.00125,90.761345,56.423814,22.086283
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3,55.743238,0.664614,0.660475,0.656336,0.00170,90.867089,56.568700,22.270310


In [17]:
def generate_signals(ohlc_data):
    # Create a new column 'Signal' with initial values as 0
    ohlc_data['Signal'] = 0
    
    # Define conditions and set signals accordingly
    bullish_condition = (ohlc_data['RSI'] >= ohlc_data['RSI_UB']) 
    
    bearish_condition = (ohlc_data['RSI'] <= ohlc_data['RSI_LB']) 
    
    ohlc_data.loc[bullish_condition, 'Signal'] = 1
    ohlc_data.loc[bearish_condition, 'Signal'] = -1
    
    return ohlc_data

In [19]:
ad = generate_signals(ad)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


In [20]:
ad

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Timestamp,Open,High,Low,Close,Volume,Alias.Underlying.RIC,RSI,Upper_Band,Middle_Band,Lower_Band,MOM,RSI_UB,RSI_MB,RSI_LB,Signal
0,2021-03-14 22:30:00,0.77600,0.77615,0.77495,0.77495,58,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2021-03-14 23:00:00,0.77560,0.77560,0.77560,0.77560,7,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2021-03-14 23:30:00,0.77525,0.77605,0.77525,0.77585,53,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2021-03-15 00:00:00,0.77610,0.77610,0.77590,0.77610,17,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2021-03-15 01:00:00,0.77690,0.77700,0.77630,0.77630,29,ADH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12350,2023-03-10 22:00:00,0.65770,0.65800,0.65770,0.65800,12,ADH3,27.222218,0.663573,0.659915,0.656257,-0.00410,90.769799,56.535307,22.300816,0
12351,2023-03-12 22:30:00,0.65960,0.66290,0.65960,0.66290,13,ADH3,51.838230,0.663970,0.660120,0.656270,0.00050,90.793162,56.788510,22.783857,0
12352,2023-03-12 23:00:00,0.66180,0.66180,0.66180,0.66180,1,ADH3,45.551597,0.664144,0.660253,0.656361,-0.00125,90.761345,56.423814,22.086283,0
12353,2023-03-12 23:30:00,0.66260,0.66480,0.66215,0.66365,159,ADH3,55.743238,0.664614,0.660475,0.656336,0.00170,90.867089,56.568700,22.270310,0


In [21]:
ad.to_csv('signal.csv')

In [22]:
df = ad

In [23]:
buy_signals_count = ad['Signal'].value_counts().get(1, 0)
sell_signals_count = ad['Signal'].value_counts().get(-1, 0)

print("Number of Buy Signals:", buy_signals_count)
print("Number of Sell Signals:", sell_signals_count)

Number of Buy Signals: 602
Number of Sell Signals: 575


In [44]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"


In [47]:
#ATR Trailing TP and SL
#Range start & stop
start = 21
stop  = len(ad) - 10

tick_size = 0.00005
tick_value = 5
tc = 1.12

def bt_tick(tp,sl, df):
    
    all_trades = []


    trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}

    
    position = None
    for i in df.index[start:stop-start]:
        if df['Signal'][i] == 1 and position!= 'Buy':  
            if trade["Symbol"] is None:
                trade["Symbol"]= "AD"
                trade["Buy/Sell"] = "Buy"
                trade["Entry"] = df["Open"][i+1] 
                trade["Entry Date"] = df["Timestamp"][i+1]

                position = "Buy"

        elif df['Signal'][i] == -1 and position!= 'Sell':
            if trade["Symbol"] is None:
                trade["Symbol"]= "AD"
                trade["Buy/Sell"] = "Sell"
                trade["Entry"] = df["Open"][i+1] 
                trade["Entry Date"] = df["Timestamp"][i+1]

                position = "Sell"

        elif position=="Buy"  and trade["Symbol"] is not None:
            if (df['Low'][i] <=trade["Entry"]- sl*tick_size):
                trade["Exit"] = trade["Entry"]- sl*tick_size 
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="SL Trigger Low"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
                
            
            elif (df["High"][i]>=trade["Entry"]+ tp*tick_size):
                trade["Exit"] = trade["Entry"]+ tp*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="TP Trigger High"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
                
            elif (df["Close"][i]<=trade["Entry"]- sl*tick_size) :
                trade["Exit"] = trade["Entry"]- sl*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="SL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None

                
            elif (df["Close"][i]>=trade["Entry"]+ tp*tick_size):
                trade["Exit"] = trade["Entry"]+ tp*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="TP Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None

        elif position=="Sell" and trade["Symbol"] is not None:
            if (df["High"][i]>=trade["Entry"]+ sl*tick_size):
                trade["Exit"] = trade["Entry"]+ sl*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="SL Trigger High"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
                
            elif (df["Low"][i]<=trade["Entry"]- tp*tick_size):
                trade["Exit"] = trade["Entry"]- tp*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="TP Trigger Low"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
                
                
            elif (df["Close"][i]>=trade["Entry"]+ sl*tick_size):
                trade["Exit"] =trade["Entry"]+ sl*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="SL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
                

            elif (df["Close"][i]<=trade["Entry"]- tp*tick_size):
                trade["Exit"] = trade["Entry"]- tp*tick_size
                trade["Exit Date"] = df["Timestamp"][i]
                trade["Detail"]="TP Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None

        


    #For Buy/Sell
    tr1= pd.DataFrame(all_trades)
    tr1["P/L"]= np.where(tr1["Buy/Sell"]=="Buy", (tr1["Exit"]-tr1["Entry"]), (tr1["Entry"]-tr1["Exit"]))
    tr1["Prob"] = 100*(np.where(tr1["P/L"]>0, 1, 0).cumsum())/(np.where(tr1["P/L"]!= np.NaN, 1,0).cumsum())
    tr1["Return"] = tr1["P/L"].cumsum()
    tr1["Entry Date"] = pd.to_datetime(tr1["Entry Date"])
    tr1["Exit Date"] = pd.to_datetime(tr1["Exit Date"])
    tr1["Duration"] = tr1["Exit Date"] - tr1["Entry Date"]
    tr1["Return per trade"] = (tr1["P/L"]/tick_size)*tick_value - tc
    tr1["Net Return"] = tr1["Return per trade"].cumsum()
    tr1["Drawdown"] = tr1["Return"] - (tr1['Return'].cummax().apply(lambda x: x if x>0 else 0))

    #For Buy only
    tr1_buy= tr1.loc[tr1["Buy/Sell"]=="Buy"]
    tr1_buy["Net Return"] = tr1_buy["Return per trade"].cumsum()
    tr1_buy["Drawdown"] = tr1_buy["Return"] - (tr1_buy['Return'].cummax().apply(lambda x: x if x>0 else 0))
    tr1_buy.reset_index(inplace=True)

    #For Sell only
    tr1_sell = tr1.loc[tr1["Buy/Sell"]== 'Sell']
    tr1_sell["Net Return"] = tr1_sell["Return per trade"].cumsum()
    tr1_sell["Drawdown"] = tr1_sell["Return"] - (tr1_sell['Return'].cummax().apply(lambda x: x if x>0 else 0))
    tr1_sell.reset_index(inplace=True)
    
    tr1.to_csv('Trade Buy_Sell.csv')
    tr1_buy.to_csv('Trade Buy.csv')
    tr1_sell.to_csv('Trade Sell.csv')

    #Trading analytics
    prob = tr1['Prob'].iloc[-1]
    win =  np.where(tr1["Return per trade"]>0, tr1["Return per trade"],0)
    total_trades = len(win)
    loss = np.where(tr1["Return per trade"]<0, tr1["Return per trade"],0)
    win = win[win!=0] 
    loss = loss[loss!=0]

    net_return = tr1["Net Return"].iloc[-1]
    total_win = np.round(np.sum(win),3)
    total_loss = np.round(np.sum(loss),3)
    total_gain = np.round(total_win + total_loss,2)
    avg_win = np.mean(win)
    avg_loss = np.mean(loss)
    max_dd = tr1["Drawdown"].min()
    max_win = np.max(win)
    max_loss = np.min(loss)
    avg_duration = np.mean(tr1["Duration"])
    risk_reward= np.abs(np.round(avg_win/avg_loss,2))
    profit_factor = np.abs(np.round(total_win/total_loss,2))


    print("--- Trading Results Buy and Sell ---\n")
    print("Total trades       : {}".format(total_trades))
    print("Wins               : {}".format(total_win))
    print("Loss               : {}".format(total_loss))
    print("Winning trades     : {}".format(len(win)))
    print("Losing trades      : {}".format(len(loss)))
    print("Average Win        : {}".format(np.round(avg_win, 5)))
    print("Average Loss       : {}".format(np.round(avg_loss, 5)))
    print("Max Win            : {}".format(np.round(max_win, 5)))
    print("Max Loss           : {}".format(np.round(max_loss, 5)))
    print("Risk Reward Ratio  : {}".format(risk_reward))
    print("Max Drawdown       : {}".format(np.round(max_dd, 5)))
    print("Profit Factor      : {}".format(profit_factor))
    print("Average Duration     {}".format(avg_duration))
    print("Win percentage       {}".format(np.round(prob,2)))
    print("---")
    print("Profit             : {} ".format(total_gain))
    print("Net Return         : {} ".format(net_return))


    
    buy= np.where(tr1["Buy/Sell"]=="Buy", tr1["Return per trade"], 0)
    buy = buy[buy!=0]
    total_trades_buy =len(buy)
    win_buy = np.where(((tr1["Buy/Sell"]=="Buy") & (tr1["Return per trade"]>0)), tr1["Return per trade"],0)
    win_buy = win_buy[win_buy!=0]
    loss_buy = np.where(((tr1["Buy/Sell"]=="Buy") & (tr1["Return per trade"]<0)), tr1["Return per trade"],0)
    loss_buy= loss_buy[loss_buy!=0]
    prob_buy = (len(win_buy)/total_trades_buy)*100
    total_buy = np.round(np.sum(buy),3)
    total_win_buy = np.round(np.sum(win_buy),3)
    total_loss_buy = np.round(np.sum(loss_buy),3)
    net_return_buy = total_win_buy + total_loss_buy
    avg_win_buy = np.mean(win_buy)
    avg_loss_buy = np.mean(loss_buy)
    max_dd_buy = tr1_buy["Drawdown"].min()
    max_win_buy = np.max(win_buy)
    max_loss_buy = np.min(loss_buy)
    avg_duration_buy = np.mean(tr1_buy['Duration'])
    risk_reward_buy = np.abs(np.round(avg_win_buy/avg_loss_buy,2))
    profit_factor_buy = np.abs(np.round(total_win_buy/total_loss_buy,2))


    print("\n--- Trading Results Only Buy ---\n")
    print("Total trades       : {}".format(total_trades_buy))
    print("Wins               : {}".format(total_win_buy))
    print("Loss               : {}".format(total_loss_buy))
    print("Winning trades     : {}".format(len(win_buy)))
    print("Losing trades      : {}".format(len(loss_buy)))
    print("Average Win        : {}".format(np.round(avg_win_buy, 5)))
    print("Average Loss       : {}".format(np.round(avg_loss_buy, 5)))
    print("Max Win            : {}".format(np.round(max_win_buy, 5)))
    print("Max Loss           : {}".format(np.round(max_loss_buy, 5)))
    print("Risk Reward Ratio  : {}".format(risk_reward_buy))
    print("Max Drawdown       : {}".format(np.round(max_dd_buy, 5)))
    print("Profit Factor      : {}".format(profit_factor_buy))
    print("Average Duration     {}".format(avg_duration_buy))
    print("Win percentage       {}".format(np.round(prob_buy,2)))
    print("---")
    print("Net Return         : {} ".format(net_return_buy))

    
    sell= np.where(tr1["Buy/Sell"]=="Sell", tr1["Return per trade"], 0)
    sell = sell[sell!=0]
    duration_sell= tr1.loc[tr1["Buy/Sell"]=="Sell", 'Duration']
    total_trades_sell =len(sell)
    win_sell = np.where(((tr1["Buy/Sell"]=="Sell") & (tr1["Return per trade"]>0)), tr1["Return per trade"],0)
    win_sell = win_sell[win_sell!=0]
    loss_sell = np.where(((tr1["Buy/Sell"]=="Sell") & (tr1["Return per trade"]<0)), tr1["Return per trade"],0)
    loss_sell= loss_sell[loss_sell!=0]
    prob_sell = (len(win_sell)/total_trades_sell)*100
    total_sell = np.round(np.sum(sell),3)
    total_win_sell = np.round(np.sum(win_sell),3)
    total_loss_sell = np.round(np.sum(loss_sell),3)
    net_return_sell = total_win_sell + total_loss_sell
    avg_win_sell = np.mean(win_sell)
    avg_loss_sell = np.mean(loss_sell)
    max_dd_sell = tr1_sell["Drawdown"].min()
    max_win_sell = np.max(win_sell)
    max_loss_sell = np.min(loss_sell)
    avg_duration_sell = np.mean(duration_sell)
    risk_reward_sell = np.abs(np.round(avg_win_buy/avg_loss_sell,2))
    profit_factor_sell = np.abs(np.round(total_win_sell/total_loss_sell,2))

    print("\n--- Trading Results Only Sell  ---\n")
    print("Total trades       : {}".format(total_trades_sell))
    print("Wins               : {}".format(total_win_sell))
    print("Loss               : {}".format(total_loss_sell))
    print("Winning trades     : {}".format(len(win_sell)))
    print("Losing trades      : {}".format(len(loss_sell)))
    print("Average Win        : {}".format(np.round(avg_win_sell, 5)))
    print("Average Loss       : {}".format(np.round(avg_loss_sell, 5)))
    print("Max Win            : {}".format(np.round(max_win_sell, 5)))
    print("Max Loss           : {}".format(np.round(max_loss_sell, 5)))
    print("Risk Reward Ratio  : {}".format(risk_reward_sell))
    print("Max Drawdown       : {}".format(np.round(max_dd_sell, 5)))
    print("Profit Factor      : {}".format(profit_factor_sell))
    print("Average Duration     {}".format(avg_duration_sell))
    print("Win percentage       {}".format(np.round(prob_sell,2)))
    print("---")
    print("Net Return         : {} ".format(net_return_sell))

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=tr1["Entry Date"],
        y=tr1['Net Return'],
        name = "Buy and Sell both" ))
    fig.add_trace(go.Scatter(
        x=tr1_buy["Entry Date"],
        y=tr1_buy['Net Return'],
        name = "Buy only" ))
    fig.add_trace(go.Scatter(
        x=tr1_sell["Entry Date"],
        y=tr1_sell['Net Return'],
        name = "Sell only" ))
    
    fig1 = px.line(df, x="Timestamp", y="Close", title='Product price over time')

    fig.show(renderer="colab")
    fig1.show(renderer="colab")

    tr1.to_csv('tr1.csv')
    return tr1


In [48]:
bt_tick(30,11,ad)

--- Trading Results Buy and Sell ---

Total trades       : 634
Wins               : 27989.44
Loss               : -25029.52
Winning trades     : 188
Losing trades      : 446
Average Win        : 148.88
Average Loss       : -56.12
Max Win            : 148.88
Max Loss           : -56.12
Risk Reward Ratio  : 2.65
Max Drawdown       : -0.0184
Profit Factor      : 1.12
Average Duration     0 days 03:14:45.804416403
Win percentage       29.65
---
Profit             : 2959.92 
Net Return         : 2959.919999996741 

--- Trading Results Only Buy ---

Total trades       : 333
Wins               : 14292.48
Loss               : -13300.44
Winning trades     : 96
Losing trades      : 237
Average Win        : 148.88
Average Loss       : -56.12
Max Win            : 148.88
Max Loss           : -56.12
Risk Reward Ratio  : 2.65
Max Drawdown       : -0.01675
Profit Factor      : 1.07
Average Duration     0 days 03:17:23.243243243
Win percentage       28.83
---
Net Return         : 992.039999999999 

---

<ipython-input-47-b34aa8bd813c>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-b34aa8bd813c>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-b34aa8bd813c>:131: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-b

,Symbol,Buy/Sell,Entry,Entry Date,Exit,Exit Date,Detail,P/L,Prob,Return,Duration,Return per trade,Net Return,Drawdown
0,AD,Buy,0.77510,2021-03-16 06:30:00,0.77455,2021-03-16 06:30:00,SL Trigger Low,-0.00055,0.000000,-0.00055,0 days 00:00:00,-56.12,-56.12,-0.00055
1,AD,Buy,0.77645,2021-03-17 19:00:00,0.77795,2021-03-17 19:00:00,TP Trigger High,0.00150,50.000000,0.00095,0 days 00:00:00,148.88,92.76,0.00000
2,AD,Buy,0.78060,2021-03-17 20:00:00,0.78210,2021-03-18 01:00:00,TP Trigger High,0.00150,66.666667,0.00245,0 days 05:00:00,148.88,241.64,0.00000
3,AD,Sell,0.78195,2021-03-18 08:00:00,0.78045,2021-03-18 08:00:00,TP Trigger Low,0.00150,75.000000,0.00395,0 days 00:00:00,148.88,390.52,0.00000
4,AD,Sell,0.78110,2021-03-18 09:30:00,0.78165,2021-03-18 10:00:00,SL Trigger High,-0.00055,60.000000,0.00340,0 days 00:30:00,-56.12,334.40,-0.00055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,AD,Buy,0.66030,2023-03-08 10:30:00,0.65975,2023-03-08 12:30:00,SL Trigger Low,-0.00055,29.523810,0.03480,0 days 02:00:00,-56.12,2774.40,-0.00295
630,AD,Buy,0.65960,2023-03-09 05:00:00,0.66110,2023-03-09 05:00:00,TP Trigger High,0.00150,29.635499,0.03630,0 days 00:00:00,148.88,2923.28,-0.00145
631,AD,Buy,0.66110,2023-03-09 06:00:00,0.66260,2023-03-09 14:00:00,TP Trigger High,0.00150,29.746835,0.03780,0 days 08:00:00,148.88,3072.16,0.00000
632,AD,Sell,0.65875,2023-03-09 21:30:00,0.65930,2023-03-09 22:00:00,SL Trigger High,-0.00055,29.699842,0.03725,0 days 00:30:00,-56.12,3016.04,-0.00055
